In [ ]:
# get rid of some chemprop warnings
# import warnings

# warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os

import pandas as pd

# load the data
df = pd.read_csv('/home/s2861704/ppk/test_kin/prep_storage/test.csv')

display(df.head())

In [ ]:
# pivot the dataframe to have the accession as columns
df = df.pivot(index="QSPRID", columns="accession", values="pchembl_value_Mean")
#df.columns.name = None
#df.reset_index(inplace=True)
print(df.columns)

display(df.head())
#Teams message + delete smiles

In [ ]:
column_list = list(df.columns)
print (column_list)

In [ ]:
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer

target_props = []
for target in column_list:
    target_props.append({'name': target, 'task': "REGRESSION"})

In [ ]:
dataset = QSPRDataset(
    name="ChempropMTDataset",
    df=df,
    target_props=target_props,
    store_dir="../../ChMT_output/data",
    random_state=42,
    # drop_empty=False,
)

dataset.getDF()

In [ ]:
from qsprpred.extra.gpu.models.chemprop import ChempropModel

ChempropModel.getAvailableParameters()

In [ ]:
# Create model
os.makedirs("../../ChMT_output/models", exist_ok=True)
model = ChempropModel(
    base_dir='../../ChMT_output/models',
    name='ChempropMTModel',
    parameters={"epochs": 5},
    quiet_logger=False
)

In [ ]:
from qsprpred.models import CrossValAssessor, TestSetAssessor
from qsprpred.models.early_stopping import EarlyStoppingMode

# We can now assess the model performance on the training set using cross validation
CrossValAssessor('r2')(model, dataset)

# and on the test set
TestSetAssessor('r2')(model, dataset)

# Finally, we need to fit the model on the complete dataset if we want to use it further
# This will save the fitted model and metadata to disk
model.earlyStopping.numEpochs = 5
model.fitDataset(dataset, mode=EarlyStoppingMode.FIXED)

# We can optionally save the model and metadata to disk explicitly as well
_ = model.save()

In [ ]:
from qsprpred.plotting.regression import CorrelationPlot

plot = CorrelationPlot([model])
axes, summary = plot.make(save=False, show=True)

In [ ]:
summary